# 📊 yfinance API 호출 제한 및 재시도 로직 테스트

이 노트북은 yfinance API의 호출 제한 문제를 분석하고 재시도 로직을 테스트합니다.

## 🎯 목표
- API 호출 제한 상황 재현 및 분석
- 재시도 로직의 효과성 검증
- PostgreSQL 연결 및 데이터 처리 테스트
- 최적의 배치 처리 방법 찾기

In [1]:
# 1. 환경 설정 및 라이브러리 임포트
import sys
import os
import time
import random
import logging
from datetime import datetime, timedelta
from typing import List, Dict, Any

# 프로젝트 경로 추가
project_root = '/home/grey1/stock-kafka3'
sys.path.insert(0, os.path.join(project_root, 'common'))
sys.path.insert(0, os.path.join(project_root, 'airflow/plugins'))

# 데이터 처리 라이브러리
import pandas as pd
import numpy as np

# yfinance 및 네트워킹 라이브러리
import yfinance as yf
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# PostgreSQL 연결
try:
    from postgresql_manager import PostgreSQLManager
    print("✅ PostgreSQL Manager 임포트 성공")
except ImportError as e:
    print(f"❌ PostgreSQL Manager 임포트 실패: {e}")

# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

print("🚀 환경 설정 완료!")
print(f"📁 프로젝트 경로: {project_root}")
print(f"🐍 Python 경로: {sys.path[:3]}")  # 처음 3개만 표시

✅ PostgreSQL Manager 임포트 성공
🚀 환경 설정 완료!
📁 프로젝트 경로: /home/grey1/stock-kafka3
🐍 Python 경로: ['/home/grey1/stock-kafka3/airflow/plugins', '/home/grey1/stock-kafka3/common', '/usr/lib/python311.zip']


In [2]:
# 1.1 필요한 패키지 설치 (curl_cffi 호환)
import subprocess
import sys

def install_package(package):
    """패키지 설치 함수"""
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"✅ {package} 설치 성공")
        return True
    except subprocess.CalledProcessError as e:
        print(f"❌ {package} 설치 실패: {e}")
        return False

# 필요한 패키지들 설치 (curl_cffi 포함)
required_packages = [
    "psycopg2-binary",   # PostgreSQL 연결용
    "yfinance",          # 주식 데이터 수집용 (최신 버전)
    "pandas",            # 데이터 처리용
    "numpy",             # 수치 계산용
    "requests",          # HTTP 요청용
    "sqlalchemy",        # SQL 인터페이스
    "curl_cffi"          # yfinance 최신 버전에 필요
]

print("📦 필요한 패키지들을 설치합니다...")
print("🔄 yfinance 최신 버전과 curl_cffi를 포함하여 설치 중...")

for package in required_packages:
    install_package(package)

# yfinance 최신 버전으로 업그레이드
print("\n🔄 yfinance를 최신 버전으로 업그레이드...")
try:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", "yfinance"])
    print("✅ yfinance 업그레이드 성공")
except subprocess.CalledProcessError as e:
    print(f"❌ yfinance 업그레이드 실패: {e}")

print("\n🔄 패키지 설치 후 다시 임포트를 시도합니다...")

# 설치 후 다시 임포트 시도
try:
    import psycopg2
    print("✅ psycopg2 임포트 성공!")
except ImportError as e:
    print(f"❌ psycopg2 임포트 여전히 실패: {e}")

try:
    import yfinance as yf
    print("✅ yfinance 임포트 성공!")
    print(f"📦 yfinance 버전: {yf.__version__}")
except ImportError as e:
    print(f"❌ yfinance 임포트 실패: {e}")

try:
    import curl_cffi
    print("✅ curl_cffi 임포트 성공!")
except ImportError as e:
    print(f"❌ curl_cffi 임포트 실패: {e}")

print("📦 패키지 설치 완료!")
print("💡 이제 yfinance가 자체적으로 curl_cffi 세션을 관리합니다.")

📦 필요한 패키지들을 설치합니다...
🔄 yfinance 최신 버전과 curl_cffi를 포함하여 설치 중...
✅ psycopg2-binary 설치 성공
✅ yfinance 설치 성공
✅ pandas 설치 성공
✅ numpy 설치 성공
✅ requests 설치 성공
✅ sqlalchemy 설치 성공
✅ curl_cffi 설치 성공

🔄 yfinance를 최신 버전으로 업그레이드...
✅ yfinance 업그레이드 성공

🔄 패키지 설치 후 다시 임포트를 시도합니다...
✅ psycopg2 임포트 성공!
✅ yfinance 임포트 성공!
📦 yfinance 버전: 0.2.65
✅ curl_cffi 임포트 성공!
📦 패키지 설치 완료!
💡 이제 yfinance가 자체적으로 curl_cffi 세션을 관리합니다.


In [3]:
# 1.2 PostgreSQL Manager 재임포트
try:
    # 프로젝트 경로 재설정
    import sys
    import os
    
    project_root = '/home/grey1/stock-kafka3'
    common_path = os.path.join(project_root, 'common')
    plugins_path = os.path.join(project_root, 'airflow/plugins')
    
    # 경로가 없으면 추가
    if common_path not in sys.path:
        sys.path.insert(0, common_path)
    if plugins_path not in sys.path:
        sys.path.insert(0, plugins_path)
        
    print(f"📁 Common 경로: {common_path}")
    print(f"📁 Plugins 경로: {plugins_path}")
    
    # PostgreSQL Manager 임포트 재시도
    from postgresql_manager import PostgreSQLManager
    print("✅ PostgreSQL Manager 임포트 성공!")
    
    # 다른 필요한 라이브러리들도 재임포트
    import yfinance as yf
    import pandas as pd
    import numpy as np
    import requests
    import time
    import random
    import logging
    from datetime import datetime, timedelta
    from typing import List, Dict, Any
    
    print("✅ 모든 라이브러리 임포트 성공!")
    
except ImportError as e:
    print(f"❌ 임포트 실패: {e}")
    
    # 파일 존재 확인
    postgresql_manager_path = '/home/grey1/stock-kafka3/common/postgresql_manager.py'
    if os.path.exists(postgresql_manager_path):
        print(f"✅ PostgreSQL Manager 파일 존재 확인: {postgresql_manager_path}")
    else:
        print(f"❌ PostgreSQL Manager 파일 없음: {postgresql_manager_path}")
        
    print(f"🔍 현재 sys.path: {sys.path[:5]}")  # 처음 5개만 표시

📁 Common 경로: /home/grey1/stock-kafka3/common
📁 Plugins 경로: /home/grey1/stock-kafka3/airflow/plugins
✅ PostgreSQL Manager 임포트 성공!
✅ 모든 라이브러리 임포트 성공!


In [4]:
# 1.3 추가 패키지 설치 및 최종 설정
import subprocess
import sys

# SQLAlchemy 및 curl_cffi 설치
try:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "sqlalchemy", "curl_cffi"])
    print("✅ sqlalchemy, curl_cffi 설치 성공")
except subprocess.CalledProcessError as e:
    print(f"❌ 패키지 설치 실패: {e}")

# 최종 임포트 시도
try:
    from postgresql_manager import PostgreSQLManager
    print("✅ PostgreSQL Manager 임포트 성공!")
    
    # API 테스터 클래스 정의 (curl_cffi 호환)
    class YFinanceAPITester:
        """yfinance API 호출 제한 및 재시도 로직 테스트 클래스 (curl_cffi 호환)"""
        
        def __init__(self):
            # curl_cffi를 위해 세션 설정 제거
            self.call_count = 0
            self.success_count = 0
            self.error_count = 0
            self.rate_limit_count = 0
            
        def test_single_symbol(self, symbol: str, period: str = "1mo", max_retries: int = 3):
            """단일 심볼에 대한 API 호출 테스트 (curl_cffi 호환)"""
            import yfinance as yf
            import time
            import random
            
            print(f"\n🔍 {symbol} 테스트 시작 (period: {period})")
            
            for attempt in range(max_retries + 1):
                try:
                    self.call_count += 1
                    start_time = time.time()
                    
                    # 랜덤 지연 추가 (1-3초)
                    delay = random.uniform(1.0, 3.0)
                    time.sleep(delay)
                    
                    # yfinance 호출 (세션 설정 제거 - yfinance가 자체 처리)
                    ticker = yf.Ticker(symbol)
                    hist = ticker.history(period=period, auto_adjust=True, prepost=False)
                    
                    end_time = time.time()
                    duration = end_time - start_time
                    
                    if not hist.empty:
                        self.success_count += 1
                        print(f"  ✅ {symbol}: 성공 ({len(hist)}일 데이터, {duration:.2f}초)")
                        return {
                            'symbol': symbol,
                            'status': 'success',
                            'data_count': len(hist),
                            'duration': duration,
                            'attempts': attempt + 1
                        }
                    else:
                        print(f"  ⚠️ {symbol}: 데이터 없음")
                        
                except Exception as e:
                    error_msg = str(e)
                    print(f"  ❌ {symbol}: 시도 {attempt + 1}/{max_retries + 1} 실패 - {error_msg}")
                    
                    # Rate limit 감지
                    if "429" in error_msg or "Too Many Requests" in error_msg or "rate limit" in error_msg.lower():
                        self.rate_limit_count += 1
                        if attempt < max_retries:
                            wait_time = (2 ** attempt) * 10 + random.uniform(5, 15)  # 지수 백오프
                            print(f"  ⏳ Rate limit 감지 - {wait_time:.1f}초 대기 후 재시도...")
                            time.sleep(wait_time)
                            continue
                    
                    self.error_count += 1
                    
                    if attempt < max_retries:
                        wait_time = 2 ** attempt * 5  # 일반 재시도
                        print(f"  ⏳ {wait_time}초 대기 후 재시도...")
                        time.sleep(wait_time)
            
            return {
                'symbol': symbol,
                'status': 'failed',
                'data_count': 0,
                'duration': 0,
                'attempts': max_retries + 1
            }
        
        def get_stats(self):
            """현재까지의 통계 반환"""
            return {
                'total_calls': self.call_count,
                'success_count': self.success_count,
                'error_count': self.error_count,
                'rate_limit_count': self.rate_limit_count,
                'success_rate': self.success_count / self.call_count * 100 if self.call_count > 0 else 0
            }
    
    # 글로벌 테스터 인스턴스 생성
    tester = YFinanceAPITester()
    print("✅ curl_cffi 호환 API 테스터 클래스 준비 완료!")
    
except Exception as e:
    print(f"❌ 최종 설정 실패: {e}")
    import traceback
    print(f"📜 상세 오류:\n{traceback.format_exc()}")

✅ sqlalchemy, curl_cffi 설치 성공
✅ PostgreSQL Manager 임포트 성공!
✅ curl_cffi 호환 API 테스터 클래스 준비 완료!


In [5]:
# 2. API 호출 제한 시뮬레이션 및 테스트 클래스 (curl_cffi 호환)
class YFinanceAPITester:
    """yfinance API 호출 제한 및 재시도 로직 테스트 클래스 (curl_cffi 호환)"""
    
    def __init__(self):
        # curl_cffi 호환을 위해 requests 세션 설정 제거
        # yfinance가 자체적으로 curl_cffi 세션을 관리하도록 함
        self.call_count = 0
        self.success_count = 0
        self.error_count = 0
        self.rate_limit_count = 0
        
    def test_single_symbol(self, symbol: str, period: str = "1mo", max_retries: int = 3):
        """단일 심볼에 대한 API 호출 테스트"""
        print(f"\n🔍 {symbol} 테스트 시작 (period: {period})")
        
        for attempt in range(max_retries + 1):
            try:
                self.call_count += 1
                start_time = time.time()
                
                # 랜덤 지연 추가 (1-3초)
                delay = random.uniform(1.0, 3.0)
                time.sleep(delay)
                
                # yfinance 호출 (세션 설정 없이)
                ticker = yf.Ticker(symbol)  # session 파라미터 제거
                hist = ticker.history(period=period, auto_adjust=True, prepost=False)
                
                end_time = time.time()
                duration = end_time - start_time
                
                if not hist.empty:
                    self.success_count += 1
                    print(f"  ✅ {symbol}: 성공 ({len(hist)}일 데이터, {duration:.2f}초)")
                    return {
                        'symbol': symbol,
                        'status': 'success',
                        'data_count': len(hist),
                        'duration': duration,
                        'attempts': attempt + 1
                    }
                else:
                    print(f"  ⚠️ {symbol}: 데이터 없음")
                    
            except Exception as e:
                error_msg = str(e)
                print(f"  ❌ {symbol}: 시도 {attempt + 1}/{max_retries + 1} 실패 - {error_msg}")
                
                # Rate limit 감지
                if "429" in error_msg or "Too Many Requests" in error_msg or "rate limit" in error_msg.lower():
                    self.rate_limit_count += 1
                    if attempt < max_retries:
                        wait_time = (2 ** attempt) * 10 + random.uniform(5, 15)  # 지수 백오프
                        print(f"  ⏳ Rate limit 감지 - {wait_time:.1f}초 대기 후 재시도...")
                        time.sleep(wait_time)
                        continue
                
                self.error_count += 1
                
                if attempt < max_retries:
                    wait_time = 2 ** attempt * 5  # 일반 재시도
                    print(f"  ⏳ {wait_time}초 대기 후 재시도...")
                    time.sleep(wait_time)
        
        return {
            'symbol': symbol,
            'status': 'failed',
            'data_count': 0,
            'duration': 0,
            'attempts': max_retries + 1
        }
    
    def test_multiple_symbols(self, symbols: List[str], period: str = "1mo"):
        """여러 심볼에 대한 배치 테스트"""
        print(f"\n🚀 배치 테스트 시작: {len(symbols)}개 심볼")
        print(f"📊 심볼 리스트: {symbols}")
        
        results = []
        start_time = time.time()
        
        for i, symbol in enumerate(symbols, 1):
            print(f"\n📈 진행률: {i}/{len(symbols)} ({i/len(symbols)*100:.1f}%)")
            result = self.test_single_symbol(symbol, period)
            results.append(result)
            
            # 배치 간 지연
            if i < len(symbols):
                batch_delay = random.uniform(2.0, 5.0)
                print(f"  💤 배치 간 대기: {batch_delay:.1f}초")
                time.sleep(batch_delay)
        
        end_time = time.time()
        total_duration = end_time - start_time
        
        # 결과 분석
        successful = [r for r in results if r['status'] == 'success']
        failed = [r for r in results if r['status'] == 'failed']
        
        print(f"\n📊 배치 테스트 결과:")
        print(f"  ⏱️ 총 소요 시간: {total_duration:.2f}초")
        print(f"  ✅ 성공: {len(successful)}개")
        print(f"  ❌ 실패: {len(failed)}개")
        print(f"  📈 성공률: {len(successful)/len(symbols)*100:.1f}%")
        print(f"  🚫 Rate limit 발생: {self.rate_limit_count}회")
        
        return {
            'results': results,
            'summary': {
                'total_symbols': len(symbols),
                'successful': len(successful),
                'failed': len(failed),
                'success_rate': len(successful)/len(symbols)*100,
                'total_duration': total_duration,
                'rate_limit_count': self.rate_limit_count,
                'total_calls': self.call_count
            }
        }
    
    def get_stats(self):
        """현재까지의 통계 반환"""
        return {
            'total_calls': self.call_count,
            'success_count': self.success_count,
            'error_count': self.error_count,
            'rate_limit_count': self.rate_limit_count,
            'success_rate': self.success_count / self.call_count * 100 if self.call_count > 0 else 0
        }

# 테스터 인스턴스 생성 (curl_cffi 호환)
print("🔄 curl_cffi 호환 API 테스터 클래스 생성 중...")
tester = YFinanceAPITester()
print("✅ curl_cffi 호환 API 테스터 클래스 준비 완료!")

🔄 curl_cffi 호환 API 테스터 클래스 생성 중...
✅ curl_cffi 호환 API 테스터 클래스 준비 완료!


In [6]:
# 3. PostgreSQL 연결 테스트
def test_postgresql_connection():
    """PostgreSQL 연결 및 심볼 데이터 조회 테스트"""
    print("🔗 PostgreSQL 연결 테스트 시작...")
    
    try:
        # 로컬 환경에서 연결 (localhost 사용)
        db = PostgreSQLManager(
            host="localhost",  # 노트북에서는 localhost 사용
            port=5432,
            database="airflow",
            user="airflow",
            password="airflow"
        )
        
        print("✅ PostgreSQL 연결 성공!")
        
        # 심볼 데이터 조회
        symbols = db.get_active_symbols()
        print(f"📊 저장된 심볼 수: {len(symbols)}개")
        
        if symbols:
            print(f"🔍 첫 10개 심볼: {symbols[:10]}")
            
            # 첫 번째 심볼의 최신 날짜 확인
            first_symbol = symbols[0]
            latest_date = db.get_latest_date(first_symbol)
            print(f"📅 {first_symbol} 최신 데이터 날짜: {latest_date}")
            
            # 기존 데이터 개수 확인
            existing_dates = db.get_existing_dates(first_symbol, days_back=30)
            print(f"📈 {first_symbol} 최근 30일 데이터: {len(existing_dates)}개")
            
        db.close()
        return symbols[:20] if symbols else []  # 테스트용으로 최대 20개만 반환
        
    except Exception as e:
        print(f"❌ PostgreSQL 연결 실패: {e}")
        print("⚠️ Docker 컨테이너가 실행 중인지 확인하세요:")
        print("   docker-compose ps")
        return []

# PostgreSQL 연결 테스트 실행
test_symbols = test_postgresql_connection()

🔗 PostgreSQL 연결 테스트 시작...
✅ PostgreSQL 테이블 생성 완료
✅ PostgreSQL 연결 성공!
📊 저장된 심볼 수: 6984개
🔍 첫 10개 심볼: ['A', 'AA', 'AACB', 'AACBR', 'AACBU', 'AACG', 'AACI', 'AACIU', 'AACIW', 'AACT']
📅 A 최신 데이터 날짜: None
📈 A 최근 30일 데이터: 0개
✅ PostgreSQL 테이블 생성 완료
✅ PostgreSQL 연결 성공!
📊 저장된 심볼 수: 6984개
🔍 첫 10개 심볼: ['A', 'AA', 'AACB', 'AACBR', 'AACBU', 'AACG', 'AACI', 'AACIU', 'AACIW', 'AACT']
📅 A 최신 데이터 날짜: None
📈 A 최근 30일 데이터: 0개


In [9]:
# 4. 기본 API 테스트 - 단일 심볼
def basic_api_test():
    """기본적인 API 호출 테스트"""
    print("🧪 기본 API 테스트 시작...")
    
    # 안정적인 심볼들로 테스트
    test_symbols_basic = ['AAPL', 'MSFT', 'GOOGL']
    
    for symbol in test_symbols_basic:
        print(f"\n📊 {symbol} 테스트:")
        result = tester.test_single_symbol(symbol, period="2y")
        print(f"   결과: {result}")

# 기본 테스트 실행
basic_api_test()

# 현재 통계 출력
stats = tester.get_stats()
print(f"\n📈 현재 통계: {stats}")

🧪 기본 API 테스트 시작...

📊 AAPL 테스트:

🔍 AAPL 테스트 시작 (period: 2y)
  ✅ AAPL: 성공 (502일 데이터, 1.64초)
   결과: {'symbol': 'AAPL', 'status': 'success', 'data_count': 502, 'duration': 1.6387951374053955, 'attempts': 1}

📊 MSFT 테스트:

🔍 MSFT 테스트 시작 (period: 2y)
  ✅ AAPL: 성공 (502일 데이터, 1.64초)
   결과: {'symbol': 'AAPL', 'status': 'success', 'data_count': 502, 'duration': 1.6387951374053955, 'attempts': 1}

📊 MSFT 테스트:

🔍 MSFT 테스트 시작 (period: 2y)
  ✅ MSFT: 성공 (502일 데이터, 3.30초)
   결과: {'symbol': 'MSFT', 'status': 'success', 'data_count': 502, 'duration': 3.302546977996826, 'attempts': 1}

📊 GOOGL 테스트:

🔍 GOOGL 테스트 시작 (period: 2y)
  ✅ MSFT: 성공 (502일 데이터, 3.30초)
   결과: {'symbol': 'MSFT', 'status': 'success', 'data_count': 502, 'duration': 3.302546977996826, 'attempts': 1}

📊 GOOGL 테스트:

🔍 GOOGL 테스트 시작 (period: 2y)
  ✅ GOOGL: 성공 (502일 데이터, 3.05초)
   결과: {'symbol': 'GOOGL', 'status': 'success', 'data_count': 502, 'duration': 3.047825574874878, 'attempts': 1}

📈 현재 통계: {'total_calls': 11, 'success_count': 6, 'er

In [ ]:
result

In [ ]:
# 5. 다양한 Period 설정 테스트 (curl_cffi 호환)
def test_different_periods():
    """다양한 기간 설정에 대한 API 응답 테스트"""
    print("📅 다양한 Period 설정 테스트...")
    
    periods = ["1mo", "3mo", "6mo", "1y", "2y"]
    test_symbol = "AAPL"
    
    results = []
    
    for period in periods:
        print(f"\n🔍 Period: {period}")
        start_time = time.time()
        
        try:
            # curl_cffi 호환을 위해 세션 설정 제거
            ticker = yf.Ticker(test_symbol)  # session 파라미터 제거
            hist = ticker.history(period=period, auto_adjust=True, prepost=False)
            
            end_time = time.time()
            duration = end_time - start_time
            
            result = {
                'period': period,
                'data_count': len(hist),
                'duration': duration,
                'status': 'success' if not hist.empty else 'empty',
                'start_date': hist.index[0].strftime('%Y-%m-%d') if not hist.empty else None,
                'end_date': hist.index[-1].strftime('%Y-%m-%d') if not hist.empty else None
            }
            
            print(f"  ✅ 데이터 수: {len(hist)}개, 소요시간: {duration:.2f}초")
            if not hist.empty:
                print(f"  📅 기간: {result['start_date']} ~ {result['end_date']}")
            
        except Exception as e:
            result = {
                'period': period,
                'data_count': 0,
                'duration': 0,
                'status': 'error',
                'error': str(e)
            }
            print(f"  ❌ 오류: {e}")
        
        results.append(result)
        
        # API 호출 간 지연
        time.sleep(random.uniform(2, 4))
    
    return results

# Period 테스트 실행
period_results = test_different_periods()

# 결과 요약
print("\n📊 Period 테스트 결과 요약:")
for result in period_results:
    status_emoji = "✅" if result['status'] == 'success' else "❌"
    print(f"  {status_emoji} {result['period']:>3}: {result['data_count']:>4}개 데이터, {result['duration']:>5.2f}초")

In [ ]:
# 6. 배치 처리 스트레스 테스트
def stress_test_batch_processing():
    """API 호출 제한을 유발하는 스트레스 테스트"""
    print("⚡ 배치 처리 스트레스 테스트 시작...")
    print("🚨 이 테스트는 의도적으로 API 호출 제한을 유발할 수 있습니다!")
    
    # 테스트할 심볼들 (DB에서 가져왔거나 기본값 사용)
    if test_symbols:
        batch_symbols = test_symbols[:10]  # DB에서 가져온 심볼 사용
        print(f"📊 DB에서 가져온 심볼 사용: {batch_symbols}")
    else:
        batch_symbols = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'META', 'NVDA', 'NFLX', 'ADBE', 'CRM']
        print(f"📊 기본 심볼 사용: {batch_symbols}")
    
    # 사용자 확인
    print(f"\n⚠️ {len(batch_symbols)}개 심볼로 스트레스 테스트를 시작합니다.")
    print("이 테스트는 API 호출 제한을 의도적으로 유발하여 재시도 로직을 테스트합니다.")
    
    # 배치 테스트 실행
    batch_result = tester.test_multiple_symbols(batch_symbols, period="2y")
    
    return batch_result

# 스트레스 테스트 실행 (사용자가 원할 때만)
print("📝 스트레스 테스트를 실행하려면 아래 주석을 해제하세요:")
print("# batch_test_result = stress_test_batch_processing()")

# 주석을 해제하면 실행됩니다:
# batch_test_result = stress_test_batch_processing()

In [ ]:
# 7. 재시도 로직 분석 및 최적화 제안
def analyze_retry_strategy():
    """현재 재시도 전략 분석 및 개선 방안 제시"""
    print("🔬 재시도 로직 분석...")
    
    # 현재 재시도 전략 시뮬레이션
    retry_strategies = {
        "현재 전략": {
            "base_delay": 20,  # 기본 대기 시간
            "max_retries": 3,
            "backoff_factor": 2,
            "description": "현재 Airflow DAG에서 사용하는 전략"
        },
        "개선된 전략": {
            "base_delay": 10,  # 더 짧은 기본 대기
            "max_retries": 5,  # 더 많은 재시도
            "backoff_factor": 1.5,  # 더 완만한 증가
            "description": "더 효율적인 재시도 전략"
        },
        "보수적 전략": {
            "base_delay": 30,  # 긴 기본 대기
            "max_retries": 2,  # 적은 재시도
            "backoff_factor": 3,  # 급격한 증가
            "description": "API 제한을 최대한 피하는 전략"
        }
    }
    
    print("\n📊 재시도 전략 비교:")
    for name, strategy in retry_strategies.items():
        delays = []
        total_time = 0
        
        print(f"\n🔹 {name}: {strategy['description']}")
        print(f"   📋 설정: 기본 {strategy['base_delay']}초, 최대 {strategy['max_retries']}회, 배수 {strategy['backoff_factor']}")
        
        for retry in range(strategy['max_retries']):
            delay = strategy['base_delay'] * (strategy['backoff_factor'] ** retry)
            delays.append(delay)
            total_time += delay
            print(f"   ⏳ {retry + 1}회 재시도: {delay:.1f}초")
        
        print(f"   🕒 총 대기 시간: {total_time:.1f}초")
        
    print("\n💡 권장사항:")
    print("1. 🔄 초기 재시도 간격을 줄이고 재시도 횟수를 늘려 성공률 향상")
    print("2. 📈 점진적 백오프로 API 서버 부하 분산")
    print("3. 🎯 Random jitter 추가로 동시 요청 충돌 방지")
    print("4. ⚡ 빠른 실패와 회로차단기 패턴 적용")

# 재시도 전략 분석 실행
analyze_retry_strategy()

# 최종 통계 출력
print("\n" + "="*50)
print("📈 최종 테스트 통계")
print("="*50)
final_stats = tester.get_stats()
for key, value in final_stats.items():
    print(f"{key:>20}: {value}")

In [ ]:
# 8. 실제 Airflow 플러그인 코드 테스트
def test_airflow_plugin():
    """실제 Airflow 플러그인의 YFinanceCollector 클래스 테스트"""
    print("🛠️ Airflow 플러그인 YFinanceCollector 테스트...")
    
    try:
        # Airflow 플러그인 임포트
        from collect_stock_data_yfinance import YFinanceCollector
        
        print("✅ YFinanceCollector 임포트 성공")
        
        # PostgreSQL 연결 (노트북 환경에서는 localhost 사용)
        collector = YFinanceCollector(
            host="localhost",
            port=5432,
            database="airflow", 
            user="airflow",
            password="airflow"
        )
        
        print("✅ YFinanceCollector 인스턴스 생성 성공")
        
        # 테스트할 심볼 (소수만)
        test_symbols_small = ['AAPL', 'MSFT']
        
        print(f"🧪 소규모 테스트 시작: {test_symbols_small}")
        
        # collect_all_symbols 메서드 테스트 (2y period)
        result = collector.collect_all_symbols(
            symbols=test_symbols_small, 
            max_workers=1,  # 안전을 위해 1개 워커만 사용
            period="2y"
        )
        
        print("✅ collect_all_symbols 메서드 테스트 완료")
        print(f"📊 결과: {result}")
        
        # 개별 심볼 테스트
        print("\n🔍 개별 심볼 테스트:")
        for symbol in test_symbols_small:
            success = collector.collect_stock_data(symbol, period="1mo")
            print(f"  {symbol}: {'✅ 성공' if success else '❌ 실패'}")
        
        collector.close()
        return result
        
    except ImportError as e:
        print(f"❌ Airflow 플러그인 임포트 실패: {e}")
        print("💡 sys.path 확인 또는 플러그인 파일 경로를 확인하세요")
        return None
    except Exception as e:
        print(f"❌ 테스트 중 오류 발생: {e}")
        import traceback
        print(f"📜 상세 오류:\n{traceback.format_exc()}")
        return None

# Airflow 플러그인 테스트 실행 (선택적)
print("📝 Airflow 플러그인 테스트를 실행하려면 아래 주석을 해제하세요:")
print("# airflow_test_result = test_airflow_plugin()")

# 주석을 해제하면 실행됩니다:
# airflow_test_result = test_airflow_plugin()

## 🎯 결론 및 해결책

### 📊 API 호출 제한 문제 분석

1. **🚫 Rate Limiting 원인**:
   - yfinance API는 Yahoo Finance의 비공식 API를 사용
   - 단시간 내 많은 요청 시 429 에러 발생
   - IP 기반 제한으로 동일 서버에서 여러 프로세스 실행 시 충돌

2. **⏱️ 현재 재시도 전략의 문제점**:
   - 20초 → 40초 → 80초 대기는 너무 긺
   - 3회 재시도 후 포기는 성공률 저하
   - 고정된 대기 시간으로 비효율적

### 💡 권장 해결책

#### 1. **재시도 로직 개선**
```python
# 현재: 20초 → 40초 → 80초
# 개선: 5초 → 10초 → 20초 → 40초 → 60초 (최대 5회)
wait_time = min(60, (2 ** attempt) * 5 + random.uniform(0, 5))
```

#### 2. **배치 크기 최적화**
- 현재: 5개 심볼씩 처리
- 권장: 2-3개 심볼씩 처리
- 심볼 간 3-5초 대기

#### 3. **API 호출 분산**
- 프록시 서버 사용
- 여러 User-Agent 로테이션
- 시간대별 API 호출 분산

#### 4. **대안 API 고려**
- Alpha Vantage API (무료 티어 제한적)
- Polygon.io API (유료, 안정적)
- IEX Cloud API (무료 티어 있음)

### 🔧 즉시 적용 가능한 개선사항

1. **재시도 간격 단축**: 20초 → 10초
2. **재시도 횟수 증가**: 3회 → 5회
3. **Random jitter 추가**: 동시 요청 방지
4. **배치 크기 감소**: 5개 → 3개
5. **심볼 간 대기 시간 증가**: 2-4초 → 3-6초

# 🚨 부하 시나리오 테스트 및 장애 대응

## 실험 목적
- yfinance API 레이트 리미팅 한계점 파악
- 최적의 재시도 전략 개발
- 장애 상황 대응 코드 검증

In [ ]:
import random
import threading
import queue
from concurrent.futures import ThreadPoolExecutor, as_completed
from dataclasses import dataclass
from typing import Dict, List, Tuple
import matplotlib.pyplot as plt
import pandas as pd

@dataclass
class LoadTestResult:
    """부하 테스트 결과를 저장하는 클래스"""
    scenario_name: str
    total_requests: int
    successful_requests: int
    failed_requests: int
    rate_limited_requests: int
    avg_response_time: float
    max_response_time: float
    error_rate: float
    rate_limit_rate: float

class YFinanceLoadTester:
    """yfinance API 부하 테스트 클래스"""
    
    def __init__(self):
        self.results = []
        self.detailed_logs = []
        
    def test_single_request_with_metrics(self, symbol: str) -> Dict:
        """단일 요청 성능 측정"""
        start_time = time.time()
        
        try:
            ticker = yf.Ticker(symbol)
            data = ticker.history(period="1d")
            
            end_time = time.time()
            response_time = end_time - start_time
            
            if data.empty:
                return {
                    'status': 'failed',
                    'error': 'Empty data',
                    'response_time': response_time,
                    'symbol': symbol
                }
            
            return {
                'status': 'success',
                'response_time': response_time,
                'symbol': symbol,
                'data_rows': len(data)
            }
            
        except Exception as e:
            end_time = time.time()
            response_time = end_time - start_time
            
            error_msg = str(e)
            is_rate_limited = 'Too Many Requests' in error_msg or '429' in error_msg
            
            return {
                'status': 'rate_limited' if is_rate_limited else 'failed',
                'error': error_msg,
                'response_time': response_time,
                'symbol': symbol
            }
    
    def run_sequential_test(self, symbols: List[str], delay: float = 0) -> LoadTestResult:
        """순차적 요청 테스트"""
        print(f"🔄 순차 테스트 시작: {len(symbols)}개 심볼, {delay}초 지연")
        
        results = []
        start_time = time.time()
        
        for i, symbol in enumerate(symbols):
            if i > 0 and delay > 0:
                time.sleep(delay)
                
            result = self.test_single_request_with_metrics(symbol)
            results.append(result)
            
            print(f"  {i+1}/{len(symbols)}: {symbol} - {result['status']} ({result['response_time']:.2f}s)")
            
        end_time = time.time()
        
        return self._analyze_results(f"Sequential-{delay}s", results, end_time - start_time)
    
    def run_concurrent_test(self, symbols: List[str], max_workers: int = 5) -> LoadTestResult:
        """동시 요청 테스트"""
        print(f"🚀 동시 테스트 시작: {len(symbols)}개 심볼, {max_workers}개 스레드")
        
        results = []
        start_time = time.time()
        
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            # 모든 작업 제출
            future_to_symbol = {
                executor.submit(self.test_single_request_with_metrics, symbol): symbol 
                for symbol in symbols
            }
            
            # 결과 수집
            for future in as_completed(future_to_symbol):
                symbol = future_to_symbol[future]
                try:
                    result = future.result()
                    results.append(result)
                    print(f"  완료: {symbol} - {result['status']} ({result['response_time']:.2f}s)")
                except Exception as e:
                    results.append({
                        'status': 'failed',
                        'error': str(e),
                        'response_time': 0,
                        'symbol': symbol
                    })
                    print(f"  예외: {symbol} - {e}")
        
        end_time = time.time()
        
        return self._analyze_results(f"Concurrent-{max_workers}threads", results, end_time - start_time)
    
    def run_burst_test(self, symbols: List[str], burst_size: int = 10, burst_interval: float = 30) -> LoadTestResult:
        """버스트 패턴 테스트"""
        print(f"💥 버스트 테스트 시작: {burst_size}개씩 {burst_interval}초 간격")
        
        results = []
        start_time = time.time()
        
        for i in range(0, len(symbols), burst_size):
            burst_symbols = symbols[i:i+burst_size]
            
            print(f"  버스트 {i//burst_size + 1}: {len(burst_symbols)}개 심볼")
            
            # 버스트 내에서는 동시 실행
            with ThreadPoolExecutor(max_workers=burst_size) as executor:
                futures = [
                    executor.submit(self.test_single_request_with_metrics, symbol)
                    for symbol in burst_symbols
                ]
                
                for future in as_completed(futures):
                    result = future.result()
                    results.append(result)
                    print(f"    {result['symbol']}: {result['status']}")
            
            # 다음 버스트까지 대기
            if i + burst_size < len(symbols):
                print(f"  {burst_interval}초 대기...")
                time.sleep(burst_interval)
        
        end_time = time.time()
        
        return self._analyze_results(f"Burst-{burst_size}x{burst_interval}s", results, end_time - start_time)
    
    def _analyze_results(self, scenario_name: str, results: List[Dict], total_time: float) -> LoadTestResult:
        """테스트 결과 분석"""
        total_requests = len(results)
        successful_requests = len([r for r in results if r['status'] == 'success'])
        failed_requests = len([r for r in results if r['status'] == 'failed'])
        rate_limited_requests = len([r for r in results if r['status'] == 'rate_limited'])
        
        response_times = [r['response_time'] for r in results]
        avg_response_time = sum(response_times) / len(response_times) if response_times else 0
        max_response_time = max(response_times) if response_times else 0
        
        error_rate = (failed_requests + rate_limited_requests) / total_requests * 100
        rate_limit_rate = rate_limited_requests / total_requests * 100
        
        result = LoadTestResult(
            scenario_name=scenario_name,
            total_requests=total_requests,
            successful_requests=successful_requests,
            failed_requests=failed_requests,
            rate_limited_requests=rate_limited_requests,
            avg_response_time=avg_response_time,
            max_response_time=max_response_time,
            error_rate=error_rate,
            rate_limit_rate=rate_limit_rate
        )
        
        self.results.append(result)
        
        print(f"\n📊 {scenario_name} 결과:")
        print(f"  총 요청: {total_requests}")
        print(f"  성공: {successful_requests} ({successful_requests/total_requests*100:.1f}%)")
        print(f"  실패: {failed_requests} ({failed_requests/total_requests*100:.1f}%)")
        print(f"  레이트 리밋: {rate_limited_requests} ({rate_limit_rate:.1f}%)")
        print(f"  평균 응답시간: {avg_response_time:.2f}초")
        print(f"  최대 응답시간: {max_response_time:.2f}초")
        print(f"  전체 소요시간: {total_time:.2f}초")
        print(f"  처리량: {total_requests/total_time:.2f} req/s")
        
        return result
    
    def generate_report(self):
        """종합 리포트 생성"""
        if not self.results:
            print("❌ 테스트 결과가 없습니다.")
            return
        
        print("\n" + "="*80)
        print("📈 부하 테스트 종합 리포트")
        print("="*80)
        
        # 테이블 형태로 결과 출력
        df = pd.DataFrame([
            {
                'Scenario': r.scenario_name,
                'Total': r.total_requests,
                'Success': r.successful_requests,
                'Failed': r.failed_requests,
                'Rate Limited': r.rate_limited_requests,
                'Error Rate (%)': f"{r.error_rate:.1f}",
                'Rate Limit (%)': f"{r.rate_limit_rate:.1f}",
                'Avg Response (s)': f"{r.avg_response_time:.2f}",
                'Max Response (s)': f"{r.max_response_time:.2f}"
            }
            for r in self.results
        ])
        
        print(df.to_string(index=False))
        
        # 시각화
        self._plot_results()
        
        return df
    
    def _plot_results(self):
        """결과 시각화"""
        if len(self.results) < 2:
            return
            
        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))
        
        scenarios = [r.scenario_name for r in self.results]
        
        # 1. 성공률 비교
        success_rates = [(r.successful_requests / r.total_requests * 100) for r in self.results]
        ax1.bar(scenarios, success_rates, color='green', alpha=0.7)
        ax1.set_title('Success Rate by Scenario')
        ax1.set_ylabel('Success Rate (%)')
        ax1.tick_params(axis='x', rotation=45)
        
        # 2. 레이트 리밋 비율
        rate_limit_rates = [r.rate_limit_rate for r in self.results]
        ax2.bar(scenarios, rate_limit_rates, color='red', alpha=0.7)
        ax2.set_title('Rate Limit Rate by Scenario')
        ax2.set_ylabel('Rate Limit Rate (%)')
        ax2.tick_params(axis='x', rotation=45)
        
        # 3. 평균 응답시간
        avg_response_times = [r.avg_response_time for r in self.results]
        ax3.bar(scenarios, avg_response_times, color='blue', alpha=0.7)
        ax3.set_title('Average Response Time by Scenario')
        ax3.set_ylabel('Response Time (seconds)')
        ax3.tick_params(axis='x', rotation=45)
        
        # 4. 에러율 비교
        error_rates = [r.error_rate for r in self.results]
        ax4.bar(scenarios, error_rates, color='orange', alpha=0.7)
        ax4.set_title('Error Rate by Scenario')
        ax4.set_ylabel('Error Rate (%)')
        ax4.tick_params(axis='x', rotation=45)
        
        plt.tight_layout()
        plt.show()

# 테스터 인스턴스 생성
load_tester = YFinanceLoadTester()
print("✅ 부하 테스터 클래스 준비 완료")

In [ ]:
# 🧪 부하 시나리오 실행

# 테스트용 심볼 리스트 (실제 운영환경과 유사하게)
test_symbols_load = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'META', 'NVDA', 'NFLX', 'BABA', 'ORCL',
                     'CRM', 'ADBE', 'PYPL', 'INTC', 'AMD', 'IBM', 'UBER', 'LYFT', 'SNAP', 'TWTR']

print(f"📋 테스트 심볼 {len(test_symbols_load)}개 준비 완료")
print(f"심볼 리스트: {test_symbols_load}")

# 시나리오 1: 순차 처리 (현재 운영 방식과 유사)
print("\n" + "="*60)
print("🔄 시나리오 1: 순차 처리 테스트")
print("="*60)

# 1-1. 지연 없이 순차 처리
result1 = load_tester.run_sequential_test(test_symbols_load[:10], delay=0)

# 1-2. 1초 지연 순차 처리  
result2 = load_tester.run_sequential_test(test_symbols_load[:10], delay=1)

# 1-3. 3초 지연 순차 처리
result3 = load_tester.run_sequential_test(test_symbols_load[:10], delay=3)

In [ ]:
# 시나리오 2: 동시 처리 테스트
print("\n" + "="*60)
print("🚀 시나리오 2: 동시 처리 테스트")
print("="*60)

# 2-1. 2개 스레드 동시 처리
result4 = load_tester.run_concurrent_test(test_symbols_load[:10], max_workers=2)

# 2-2. 5개 스레드 동시 처리 (현재 실패하는 방식)
result5 = load_tester.run_concurrent_test(test_symbols_load[:10], max_workers=5)

# 2-3. 10개 스레드 동시 처리 (고부하)
result6 = load_tester.run_concurrent_test(test_symbols_load[:10], max_workers=10)

In [ ]:
# 시나리오 3: 버스트 패턴 테스트
print("\n" + "="*60)
print("💥 시나리오 3: 버스트 패턴 테스트")
print("="*60)

# 3-1. 소규모 버스트 (3개씩 30초 간격)
result7 = load_tester.run_burst_test(test_symbols_load[:12], burst_size=3, burst_interval=30)

# 3-2. 중간 버스트 (5개씩 60초 간격) 
result8 = load_tester.run_burst_test(test_symbols_load[:15], burst_size=5, burst_interval=60)

# 종합 리포트 생성
print("\n" + "="*80)
print("📊 최종 분석 리포트")
print("="*80)

final_report = load_tester.generate_report()